In [99]:
from pyspark.sql import SparkSession
import pandas as pd
from graphframes import GraphFrame
from pyspark.sql.functions import desc  
from pyspark.sql.functions import col 

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.master("local[*]") \
    .enableHiveSupport().appName("spark_app1") \
    .config('spark.jars.packages', 'graphframes:graphframes:0.7.0-spark2.4-s_2.11')\
    .getOrCreate()

spark.version   # 2.4.5

'2.4.5'

In [13]:
from graphframes import GraphFrame
from pyspark.sql.functions import desc  
v1 = spark.createDataFrame([('1', 'Carter', 'Derrick', 50), 
                                  ('2', 'May', 'Derrick', 26),
                                 ('3', 'Mills', 'Jeff', 80),
                                  ('4', 'Hood', 'Robert', 65),
                                  ('5', 'Banks', 'Mike', 93),
                                 ('98', 'Berg', 'Tim', 28),
                                 ('99', 'Page', 'Allan', 16)],
                                 ['id', 'name', 'firstname', 'age'])
e1 = spark.createDataFrame([('1', '2', 'friend'), 
                               ('2', '1', 'friend'),
                              ('3', '1', 'friend'),
                              ('1', '3', 'friend'),
                               ('2', '3', 'follows'),
                               ('3', '4', 'friend'),
                               ('4', '3', 'friend'),
                               ('5', '3', 'friend'),
                               ('3', '5', 'friend'),
                               ('4', '5', 'follows'),
                              ('98', '99', 'friend'),
                              ('99', '98', 'friend')],
                              ['id1', 'id2', 'type'])
# 컬럼명 변경하기
e1 = e1.withColumnRenamed("id1", "src").withColumnRenamed("id2", "dst")

g = GraphFrame(v1, e1)
g.vertices.show()
g.edges.show()

+---+------+---------+---+
| id|  name|firstname|age|
+---+------+---------+---+
|  1|Carter|  Derrick| 50|
|  2|   May|  Derrick| 26|
|  3| Mills|     Jeff| 80|
|  4|  Hood|   Robert| 65|
|  5| Banks|     Mike| 93|
| 98|  Berg|      Tim| 28|
| 99|  Page|    Allan| 16|
+---+------+---------+---+

+---+---+-------+
|src|dst|   type|
+---+---+-------+
|  1|  2| friend|
|  2|  1| friend|
|  3|  1| friend|
|  1|  3| friend|
|  2|  3|follows|
|  3|  4| friend|
|  4|  3| friend|
|  5|  3| friend|
|  3|  5| friend|
|  4|  5|follows|
| 98| 99| friend|
| 99| 98| friend|
+---+---+-------+



In [7]:
g.degrees.show()

+---+------+
| id|degree|
+---+------+
|  3|     7|
| 98|     2|
| 99|     2|
|  5|     3|
|  1|     4|
|  4|     3|
|  2|     3|
+---+------+



In [8]:
g.inDegrees.show()


+---+--------+
| id|inDegree|
+---+--------+
|  3|       4|
| 98|       1|
| 99|       1|
|  5|       2|
|  1|       2|
|  4|       1|
|  2|       1|
+---+--------+



In [9]:
# 출
g.outDegrees.show()

+---+---------+
| id|outDegree|
+---+---------+
|  3|        3|
| 98|        1|
| 99|        1|
|  5|        1|
|  1|        2|
|  4|        2|
|  2|        2|
+---+---------+



In [10]:
# 정점부 조건 설정
g.vertices.filter("age > 30").show()

+---+------+---------+---+
| id|  name|firstname|age|
+---+------+---------+---+
|  1|Carter|  Derrick| 50|
|  3| Mills|     Jeff| 80|
|  4|  Hood|   Robert| 65|
|  5| Banks|     Mike| 93|
+---+------+---------+---+



In [11]:
# 관계부 조건 설정
g.edges.filter('type == "friend"').show()

+---+---+------+
|src|dst|  type|
+---+---+------+
|  1|  2|friend|
|  2|  1|friend|
|  3|  1|friend|
|  1|  3|friend|
|  3|  4|friend|
|  4|  3|friend|
|  5|  3|friend|
|  3|  5|friend|
| 98| 99|friend|
| 99| 98|friend|
+---+---+------+



In [14]:
# inDegrees에서 조건을 정해서 출력
g.inDegrees.filter("inDegree >= 2").sort("inDegree", ascending=False).show()

+---+--------+
| id|inDegree|
+---+--------+
|  3|       4|
|  1|       2|
|  5|       2|
+---+--------+



In [70]:
# 서로 연결되어있는거 찾기 또는 경로를 지정해서 찾기
g.find("(a)-[e]->(b); (b)-[e2]->(c); (c)-[e3]->(a)").show(10,False)

+------------------------+---------------+------------------------+---------------+------------------------+---------------+
|a                       |e              |b                       |e2             |c                       |e3             |
+------------------------+---------------+------------------------+---------------+------------------------+---------------+
|[3, Mills, Jeff, 80]    |[3, 1, friend] |[1, Carter, Derrick, 50]|[1, 2, friend] |[2, May, Derrick, 26]   |[2, 3, follows]|
|[2, May, Derrick, 26]   |[2, 3, follows]|[3, Mills, Jeff, 80]    |[3, 1, friend] |[1, Carter, Derrick, 50]|[1, 2, friend] |
|[3, Mills, Jeff, 80]    |[3, 4, friend] |[4, Hood, Robert, 65]   |[4, 5, follows]|[5, Banks, Mike, 93]    |[5, 3, friend] |
|[1, Carter, Derrick, 50]|[1, 2, friend] |[2, May, Derrick, 26]   |[2, 3, follows]|[3, Mills, Jeff, 80]    |[3, 1, friend] |
|[4, Hood, Robert, 65]   |[4, 5, follows]|[5, Banks, Mike, 93]    |[5, 3, friend] |[3, Mills, Jeff, 80]    |[3, 4, friend] |


In [24]:
# 최단 거리 찾기 BFS(너비우선)// DFS(깊이 우선)
g.bfs(fromExpr="id='1'", toExpr="id='4'", maxPathLength=30).show(truncate=False)

+------------------------+--------------+--------------------+--------------+---------------------+
|from                    |e0            |v1                  |e1            |to                   |
+------------------------+--------------+--------------------+--------------+---------------------+
|[1, Carter, Derrick, 50]|[1, 3, friend]|[3, Mills, Jeff, 80]|[3, 4, friend]|[4, Hood, Robert, 65]|
+------------------------+--------------+--------------------+--------------+---------------------+



## Bike

In [123]:
# 데이터 불러오기
bikeStations = spark.read.option("header","true").csv("./bike-data/201508_station_data.csv")
tmp = spark.read.option("header","true").csv("./bike-data/201508_trip_data.csv")

# 데이터 프레임 구조인 경우 limit
tripData=tmp.limit(50)
tripData

DataFrame[Trip ID: string, Duration: string, Start Date: string, Start Station: string, Start Terminal: string, End Date: string, End Station: string, End Terminal: string, Bike #: string, Subscriber Type: string, Zip Code: string]

In [124]:
# 맵으로 구현
import matplotlib.pyplot as plt
import matplotlib
import folium

data = bikeStations.select("lat", "long","name").collect()
# 지도 시작시 위치, zoom_size9 지도확대 비율 ~18까지 있음.
map_osm = folium.Map(location=[37.329732,  -121.901782], zoom_size=14)

for i in data: #반복문을 회전하면서 지도위에 마커를 표시함.
    folium.Marker([float(i['lat']), float(i['long'])], popup=i['name']).add_to(map_osm)
    
map_osm

#map_osm.save('bike_map.html')

In [119]:
# 컬럼명 변경하기
stationVertices = bikeStations.withColumnRenamed("name", "id")

tripEdges = tripData.withColumnRenamed("Start Station", "src")\
                .withColumnRenamed("End Station", "dst")

stationGraph = GraphFrame(stationVertices, tripEdges)

stationGraph.vertices.show()
stationGraph.edges.show()

AttributeError: 'DataFrame' object has no attribute 'withColumnRenamed'

In [93]:
# 시작과 끝이 가장 많은 것 10개 구하기
stationGraph.edges.groupBy("src", "dst").count().orderBy(desc("count")).show(10)

+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|       5th at Howard|San Francisco Cal...|    3|
|   Market at Sansome|Broadway St at Ba...|    2|
|Embarcadero at Fo...|Embarcadero at Sa...|    2|
|San Francisco Cal...|     Townsend at 7th|    2|
|Harry Bridges Pla...|San Francisco Cal...|    1|
|  San Jose City Hall| San Salvador at 1st|    1|
|San Antonio Shopp...|Mountain View Cit...|    1|
|Yerba Buena Cente...|San Francisco Cal...|    1|
|Embarcadero at Sa...|   Steuart at Market|    1|
|      Post at Kearny|   2nd at South Park|    1|
+--------------------+--------------------+-----+
only showing top 10 rows



### 가장 많이 출발하는 지점을 찾으시오.

In [94]:
stationGraph.outDegrees.sort('outDegree', ascending = False).show()

+--------------------+---------+
|                  id|outDegree|
+--------------------+---------+
|   Market at Sansome|        4|
|San Francisco Cal...|        3|
|Embarcadero at Sa...|        3|
|       5th at Howard|        3|
|Yerba Buena Cente...|        3|
|San Francisco Cal...|        3|
|     Spear at Folsom|        3|
|Temporary Transba...|        2|
|     2nd at Townsend|        2|
|       Howard at 2nd|        2|
|      Post at Kearny|        2|
|      Market at 10th|        2|
|     Townsend at 7th|        2|
|Harry Bridges Pla...|        2|
|Embarcadero at Fo...|        2|
|San Antonio Shopp...|        1|
|  San Jose City Hall|        1|
|     Beale at Market|        1|
|University and Em...|        1|
|       Market at 4th|        1|
+--------------------+---------+
only showing top 20 rows



### 가장 많이 종료하는 지점을 찾으시오.

In [95]:
stationGraph.inDegrees.sort('inDegree', ascending = False).show()

+--------------------+--------+
|                  id|inDegree|
+--------------------+--------+
|San Francisco Cal...|       6|
|San Francisco Cal...|       6|
|Embarcadero at Sa...|       3|
|Harry Bridges Pla...|       3|
|Broadway St at Ba...|       3|
|     Townsend at 7th|       3|
|     Spear at Folsom|       2|
|   2nd at South Park|       2|
|   Steuart at Market|       2|
|Embarcadero at Br...|       2|
|     2nd at Townsend|       2|
|   Market at Sansome|       2|
|Grant Avenue at C...|       2|
|Mountain View Cit...|       1|
|Temporary Transba...|       1|
| San Salvador at 1st|       1|
|South Van Ness at...|       1|
|Castro Street and...|       1|
|Cowper at University|       1|
|       5th at Howard|       1|
+--------------------+--------+
only showing top 20 rows



### 'Townsend at 7th' 시작해서 'Spear at Folsom'로 이동하는 경로를 탐색(bfs)하시오.

In [88]:
Town7_Spear=stationGraph.bfs(fromExpr="id = 'Townsend at 7th'",toExpr="id = 'Spear at Folsom'", maxPathLength=2).show(10, False)

+---------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------+
|from                                                                       |e0                                                                                                                |to                                                                         |
+---------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------+
|[65, Townsend at 7th, 37.771058, -122.402717, 15, San Francisco, 8/22/2013]|[913371, 663, 8/31/2015 20:10, Townsend at 7th, 65, 8/31/2015 20:21, Spear at Folsom, 49, 405, Subscriber, 94103] |[

### a -> b -> c -> a로 이동하는 코스를 구하시오.

In [97]:
stationGraph.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(a)").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   a|                  e1|                   b|                  e2|                   c|                  e3|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[60, Embarcadero ...|[913443, 691, 8/3...|[77, Market at Sa...|[913403, 497, 8/3...|[61, 2nd at Towns...|[913383, 1435, 8/...|
|[69, San Francisc...|[913434, 283, 8/3...|[65, Townsend at ...|[913371, 663, 8/3...|[49, Spear at Fol...|[913432, 703, 8/3...|
|[49, Spear at Fol...|[913432, 703, 8/3...|[69, San Francisc...|[913404, 273, 8/3...|[65, Townsend at ...|[913371, 663, 8/3...|
|[49, Spear at Fol...|[913432, 703, 8/3...|[69, San Francisc...|[913434, 283, 8/3...|[65, Townsend at ...|[913371, 663, 8/3...|
|[69, San Francisc...|[913404, 273, 8/3...|[65, Townsend at ...|[913371, 663, 8/3...|[49, Spear at Fol..